In [21]:
from azureml.core import Workspace, Datastore, Dataset

from torch import nn
import torch
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, random_split

from torch.amp import autocast, GradScaler

import matplotlib.pyplot as plt

from torch import optim

In [2]:
ws = Workspace.from_config()

ds = Datastore.get(ws, "cvdatastore")

dataset = Dataset.File.from_files(path=(ds, "Plant_leaf_diseases_dataset_with_augmentation/Plant_leave_diseases_dataset_with_augmentation/**"))

mount_context = dataset.mount()
mount_context.start()

data_path = mount_context.mount_point

In [25]:
# As the data is already augmented we will go with no aug transform
transform = transforms.Compose([
    transforms.Resize((244,244)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

full_dataset = datasets.ImageFolder(root=data_path,transform=transform)

train_ratio = int(0.8*len(full_dataset))
test_ratio = len(full_dataset)-train_ratio

train_ds, test_ds = random_split(full_dataset, [train_ratio, test_ratio])


train_loader = DataLoader(train_ds, batch_size=32, shuffle=True, num_workers=6, pin_memory=True, prefetch_factor=2)
test_loader = DataLoader(test_ds, batch_size=32, shuffle=False, num_workers=6, pin_memory=True, prefetch_factor=2)

device = "cuda" if torch.cuda.is_available() else "cpu"

In [26]:
# Model & eval definition

model = models.resnet18(pretrained=True)

for param in model.parameters():
    param.requires_grad=False

model.fc = nn.Linear(model.fc.in_features, len(full_dataset.classes))

model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(),lr=1e-3)

/anaconda/envs/azureml_py38_PT_TF/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/anaconda/envs/azureml_py38_PT_TF/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [27]:
# Training 
epochs = 5
scalar = GradScaler(device)

for epoch in range(epochs):
    model.train()
    running_loss = 0.0

    for images, labels in train_loader:
        images, labels = images.to(device, non_blocking=True), labels.to(device, non_blocking=True)

        optimizer.zero_grad(set_to_none=True)
        
        with autocast(device):
            out = model(images)
            loss = criterion(out, labels)

        scalar.scale(loss).backward()
        scalar.step(optimizer)
        scalar.update()
        
        running_loss += loss.item()*images.size(0)
    epoch_loss = running_loss/len(train_loader.dataset)
    
    model.eval()
    correct, total = 0,0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device, non_blocking = True),labels.to(device, non_blocking = True)

            with autocast(device):
                out = model(images)
            
            _, pred = torch.max(out,1)
            total += labels.size(0)
            correct += (pred==labels).sum().item()

    print(f"Epoch:{epoch+1}/{epochs}\tepoch Loss:{epoch_loss}\tAccuracy:{100*correct/total}")

Epoch:1/5	epoch Loss:0.5388048978232655	Accuracy:94.68206212392259
Epoch:2/5	epoch Loss:0.18615872945554413	Accuracy:95.26752317449991
Epoch:3/5	epoch Loss:0.1449804331865254	Accuracy:95.87737843551797
Epoch:4/5	epoch Loss:0.1262207428202296	Accuracy:96.18637176776711
Epoch:5/5	epoch Loss:0.11370420669204502	Accuracy:96.49536510001626


In [8]:
mount_context.stop()

model.eval()
torch.save(model.state_dict,"leaf_disease_model.pth")

In [ ]:
!pip install scikit-learn seaborn

In [24]:
from sklearn.metrics import confusion_matrix, classification_report
import matplotlib.pyplot as plt
import seaborn as sns

# Valuation metrics
all_preds, all_labels = [], []
    
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())


conf_mat = confusion_matrix(all_labels, all_preds)

plt.figure(figsize=(12,8))
sns.heatmap(conf_mat, annot=False,cmap="Blues", xticklabels=full_dataset.classes, yticklabels=full_dataset.classes)
plt.xlabel("Predicted")
plt.ylabel("True")
plt.title("Confusion Matrix")

plt.show()


print(classification_report(all_labels, all_preds, target_names=full_dataset.classes))


ModuleNotFoundError: No module named 'sklearn'